# Baseline RAG example

This is a simple example of a baseline RAG application which purpose is to answer questions about the fantasy series [Malazan Universe](https://malazan.fandom.com/wiki/Malazan_Wiki) created by Steven Erikson and Ian C. Esslemont.

First the example will show each step of a baseline RAG pipeline including **Indexing**, **Retrieval** and **Generation**. This is done in order to show the architecture without the abstraction provided by frameworks like LlamaIndex and LangChain.
Then a more "normal" example will be shown using LlamaIndex.

As a vector database, we will use [ChromaDB](https://docs.trychroma.com/), but this can easily be exchanged with other databases.

In this example, we will use the following technologies

- OpenAI API
- ChromaDB
- LlamaIndex


### Setup libraries and environment


In [ ]:
%pip install chromadb llama-index-vector-stores-chroma

In [14]:
import os

import chromadb
import chromadb.utils.embedding_functions as embedding_functions
from chromadb import Settings
from IPython.display import Markdown, display
from llama_index.core import PromptTemplate, SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from openai import OpenAI

from util.helpers import create_and_save_md_files, get_malazan_pages

### Environment variables

For this example you need to use an OpenAI API key. Go to [your API keys](https://platform.openai.com/api-keys) in the OpenAI console to generate one.

Then add the following to a `.env` file in the root of the project.

```
OPENAI_API_KEY=<YOUR_KEY_HERE>
```


In [2]:
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
openai_client = OpenAI(api_key=OPENAI_API_KEY)
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=OPENAI_API_KEY,
    model_name="text-embedding-3-small"
)

chroma_client = chromadb.PersistentClient(
    path="./data/baseline-rag/chromadb", settings=Settings(allow_reset=True))

## Fetch documents and save them as markdown files

Here we fetch pages from the Fandom Malazan Wiki. These are the documents that we will use as our "knowledge base" in order to supply context to our prompts.

We also pre-process the content in order to be able to add them to our vector database.


In [4]:
pages = get_malazan_pages()
create_and_save_md_files(pages)

## Indexing

In this step, we will index the documents in our vector database. This will allow us to retrieve the most relevant documents when we ask a question.

We will use ChromaDB as our vector database and 'text-embedding-3-small' from OpenAI as our embedding model.


#### Fetch and process saved documents

First we need to fetch the documents we saved earlier.

Then we will process the documents in order to add them to our vector database.
The `SimpleDirectoryReader` fetches each section of the markdown file
Then each section is split in to smaller chunks of text and each chunk is embedded using the OpenAI API.


In [5]:
documents = SimpleDirectoryReader('./data/docs').load_data()
text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=20)

document_data = []

for document in documents:
    chunks = text_splitter.split_text(document.text)
    for idx, chunk in enumerate(chunks):
        embedding = openai_client.embeddings.create(
            input=chunk, model="text-embedding-3-small")
        document_data.append({
            "id": f"{document.id_}-{idx}",
            "text": chunk,
            "metadata": document.metadata,
            "embedding": embedding.data[0].embedding
        })

#### Add documents to ChromaDB


In [6]:
documents = [doc["text"] for doc in document_data]
embeddings = [doc["embedding"] for doc in document_data]
metadatas = [doc["metadata"] for doc in document_data]
ids = [doc["id"] for doc in document_data]

In [7]:
chroma_client.reset()
collection = chroma_client.get_or_create_collection(
    name="malazan", metadata={"hnsw:space": "cosine"}, embedding_function=openai_ef)

In [8]:
collection.add(
    embeddings=embeddings,
    documents=documents,
    metadatas=metadatas,
    ids=ids)

## Retrieval

In this step, we will retrieve the most relevant documents to a given question. We will use the vector database to retrieve the most similar documents to the question.

In order to do this we will use the `text-embedding-3-small` model (**the same model used to index the documents**) from OpenAI to embed the question and then use the vector database to retrieve the most similar documents.

We will retrieve the top 5 documents based on the _cosine similarity_ between the question and the documents. Other similarity metrics can be used as well like squared L2 or inner product.

Change `cosine` to `l2` or `ip` when creating the collection above to try these out.


In [9]:
query = "What was the titles of Anomander Rake?"

In [ ]:
query = "Who is Tayschrenn?"

In [ ]:
query = "What is Kurald Galain?"

In [10]:
result = collection.query(query_texts=[query], n_results=5)
context = result["documents"][0]
context
display(Markdown(f"------------\n\n{"\n\n------------\n\n".join(context)}"))

------------

Other names


Anomander Rake bore a number of pseudonyms and titles.
Anomandaris Irake
Anomandaris Purake
Anomander Dragnipurake
Black-Winged Lord
Blacksword
First Son of Darkness
Knight of Darkness
Knight of High House Dark
Lord of Moon's Spawn
The Mane of Chaos
The Rake
Son of Darkness

------------

Anomander Rake


"Anomander Rake, lord of the black-skinned Tiste Andii, who has looked down on a hundred thousand winters, who has tasted the blood of dragons, who leads the last of his kind, seated in the throne of sorrow and a kingdom tragic and fey— a kingdom with no land to call its own."
―Tattersail, upon seeing Anomander Rake for the first time
Anomander Rake also known by his titles of the Lord of Moon's Spawn, Son of Darkness and Knight of Darkness was the leader of the Tiste Andii. He was said to be "seated on the Throne of Sorrow"; a reflection of the loss and apathy his people had suffered.
Baruk described Rake as having jet-black skin, a mane which flowed silver and features, sharp as if cut from onyx. The Tiste was nearly seven feet tall and on that occasion was cloaked and wore boots. He carried the enormous two-handed sword Dragnipur on his back. There, it was "surrounded in its own breath of preternatural darkness."
Rake's eyes were described as multihued with a slight upward tilt and large vertical pupils. Baruk perceived them as changing colour from a deep hue of amber to grey and banded, a rainbow of colours changing depending on Rake's mood. Whilst with Baruk, Rake's eye colour was in turn green, dun, grey and black. When Paran encountered Rake, he describe their colour as a deep, cold blue which lightened to sky blue a short while later.
As a mage he made use of the Elder Warren of Kurald Galain. Baruk could feel the power emanating from Rake and judged him to be more powerful than the Malazan High Mage, Tayschrenn.
By the time of Gardens of the Moon, Rake disliked the title "Son of Darkness", saying it was used "by those fools who think me worthy of worship."

------------

Summary


"Anomander Rake, lord of the black-skinned Tiste Andii, who has looked down on a hundred thousand winters, who has tasted the blood of dragons, who leads the last of his kind, seated in the throne of sorrow and a kingdom tragic and fey— a kingdom with no land to call its own."

------------

Anomander Rake and the Hounds of Shadow by Marc SimonettiAnomander Rake in Dragnipur by PuckAnomander Rake and Mother Darkness by Dejan DelicAnomander Rake and Mother Dark by Mister Adam

------------

History


Spoiler warning: The following section contains significant plot details about Fall of Light and Toll the Hounds.
Anomander Purake, later named Anomander Rake by Caladan Brood, was given the title of the First Son of Darkness by Mother Dark, queen of Kharkanas and goddess of the Tiste Andii. He was a Soletaken — able to assume the form of a huge black dragon, larger even than Silanah (a true Eleint) — and a very powerful Ascendant, occupying the place of the Knight of High House Dark in the Deck of Dragons. He was also the wielder of (the sword) Vengeance/Grief, and of the dangerous and powerful Dragnipur, which was forged by his friend Draconus — on whom he ultimately turned his own creation, thereby fulfilling Kallor's curse.
He was widely recognised — by many Ascendants, gods and Elder Gods — as one of the most powerful, unpredictable and ruthless figures in the World, especially since he was the wielder of the sword that enslaved souls.
In his early years — during the events of the Kharkanas Trilogy — he seemed somewhat arrogant and rash, but in his later years — during the events of the Book of the Fallen — he was portrayed as more compassionate and wise. Rake was a man of solitude, possessing a spirit of almost pathological independence. He could be indifferent to others' need for regular reassurance or comfirmation, trusting that his word, once given, was bond enough. What Rake said he would do, he did. His thought processes were often a mystery to his own people, perhaps because of his draconic blood, but still they trusted and loved him without question.
He was the son of Nimander Purake — who in his younger years was granted, by K'rul, the Azathanai honorific Purake, which came from "Pur Rakess Calas ne A’nom", roughly translated to "Strength in Standing Still" — and had two brothers, Silchas Ruin and Andarist, both given the titles the Second Son of Darkness and the Third Son of Darkness, respectively. T'riss was a one-time companion, as were Lady Envy and Caladan Brood.

## Generation

In this step, we will generate an answer to the question using the retrieved documents as context. We will use the OpenAI API to generate the answer.


In [11]:
prompt = PromptTemplate("""You are a helpful assistant that answers questions about the Malazan Fantasy Universe using provided context. 

Question: {query}

Context: 

-----------------------------------
{context}

-----------------------------------

""")
message = prompt.format(query=query, context="\n\n".join(context))
display(Markdown(f"{message}"))

You are a helpful assistant that answers questions about the Malazan Fantasy Universe using provided context. 

Question: What was the titles of Anomander Rake?

Context: 

-----------------------------------
Other names


Anomander Rake bore a number of pseudonyms and titles.
Anomandaris Irake
Anomandaris Purake
Anomander Dragnipurake
Black-Winged Lord
Blacksword
First Son of Darkness
Knight of Darkness
Knight of High House Dark
Lord of Moon's Spawn
The Mane of Chaos
The Rake
Son of Darkness

Anomander Rake


"Anomander Rake, lord of the black-skinned Tiste Andii, who has looked down on a hundred thousand winters, who has tasted the blood of dragons, who leads the last of his kind, seated in the throne of sorrow and a kingdom tragic and fey— a kingdom with no land to call its own."
―Tattersail, upon seeing Anomander Rake for the first time
Anomander Rake also known by his titles of the Lord of Moon's Spawn, Son of Darkness and Knight of Darkness was the leader of the Tiste Andii. He was said to be "seated on the Throne of Sorrow"; a reflection of the loss and apathy his people had suffered.
Baruk described Rake as having jet-black skin, a mane which flowed silver and features, sharp as if cut from onyx. The Tiste was nearly seven feet tall and on that occasion was cloaked and wore boots. He carried the enormous two-handed sword Dragnipur on his back. There, it was "surrounded in its own breath of preternatural darkness."
Rake's eyes were described as multihued with a slight upward tilt and large vertical pupils. Baruk perceived them as changing colour from a deep hue of amber to grey and banded, a rainbow of colours changing depending on Rake's mood. Whilst with Baruk, Rake's eye colour was in turn green, dun, grey and black. When Paran encountered Rake, he describe their colour as a deep, cold blue which lightened to sky blue a short while later.
As a mage he made use of the Elder Warren of Kurald Galain. Baruk could feel the power emanating from Rake and judged him to be more powerful than the Malazan High Mage, Tayschrenn.
By the time of Gardens of the Moon, Rake disliked the title "Son of Darkness", saying it was used "by those fools who think me worthy of worship."

Summary


"Anomander Rake, lord of the black-skinned Tiste Andii, who has looked down on a hundred thousand winters, who has tasted the blood of dragons, who leads the last of his kind, seated in the throne of sorrow and a kingdom tragic and fey— a kingdom with no land to call its own."

Anomander Rake and the Hounds of Shadow by Marc SimonettiAnomander Rake in Dragnipur by PuckAnomander Rake and Mother Darkness by Dejan DelicAnomander Rake and Mother Dark by Mister Adam

History


Spoiler warning: The following section contains significant plot details about Fall of Light and Toll the Hounds.
Anomander Purake, later named Anomander Rake by Caladan Brood, was given the title of the First Son of Darkness by Mother Dark, queen of Kharkanas and goddess of the Tiste Andii. He was a Soletaken — able to assume the form of a huge black dragon, larger even than Silanah (a true Eleint) — and a very powerful Ascendant, occupying the place of the Knight of High House Dark in the Deck of Dragons. He was also the wielder of (the sword) Vengeance/Grief, and of the dangerous and powerful Dragnipur, which was forged by his friend Draconus — on whom he ultimately turned his own creation, thereby fulfilling Kallor's curse.
He was widely recognised — by many Ascendants, gods and Elder Gods — as one of the most powerful, unpredictable and ruthless figures in the World, especially since he was the wielder of the sword that enslaved souls.
In his early years — during the events of the Kharkanas Trilogy — he seemed somewhat arrogant and rash, but in his later years — during the events of the Book of the Fallen — he was portrayed as more compassionate and wise. Rake was a man of solitude, possessing a spirit of almost pathological independence. He could be indifferent to others' need for regular reassurance or comfirmation, trusting that his word, once given, was bond enough. What Rake said he would do, he did. His thought processes were often a mystery to his own people, perhaps because of his draconic blood, but still they trusted and loved him without question.
He was the son of Nimander Purake — who in his younger years was granted, by K'rul, the Azathanai honorific Purake, which came from "Pur Rakess Calas ne A’nom", roughly translated to "Strength in Standing Still" — and had two brothers, Silchas Ruin and Andarist, both given the titles the Second Son of Darkness and the Third Son of Darkness, respectively. T'riss was a one-time companion, as were Lady Envy and Caladan Brood.

-----------------------------------



In [12]:
stream = openai_client.chat.completions.create(
    messages=[{"role": "user", "content": query}],
    model="gpt-4-turbo",
    stream=True)

output = ""
for chunk in stream:
    output += chunk.choices[0].delta.content or ""
    display(Markdown(f"{output}"), clear=True)

Anomander Rake, a key character in Steven Erikson's "Malazan Book of the Fallen" series, holds several titles and is known by various names throughout the series. Below are some of his prominent titles and aliases:

1. **Son of Darkness** - This title refers to his position as a prominent member of the Tiste Andii, a race associated with darkness.
2. **Knight of Darkness** - Similar to "Son of Darkness", this title emphasizes his role and responsibilities among the Tiste Andii.
3. **Lord of Moon's Spawn** - Refers to his command over the floating fortress Moon's Spawn, which is often involved in pivotal battles in the series.
4. **Mane of Chaos** - A title that highlights his connection and contributions to the greater forces of Chaos within the Malazan universe.

Anomander Rake is also referred to by his real name, Anomandaris Irake, and is a complex character central to many events and story arcs in Erikson's expansive epic. His possession of the sword Dragnipur, which imprisons the souls of those it kills, adds to his formidable and mythical stature in the series.

In [ ]:
stream = openai_client.chat.completions.create(
    messages=[{"role": "user", "content": message}],
    model="gpt-4-turbo",
    stream=True)

output = ""
for chunk in stream:
    output += chunk.choices[0].delta.content or ""
    display(Markdown(f"{output}"), clear=True)

## Normal example using LlamaIndex

In this example, we will use LlamaIndex to abstract the indexing and retrieval steps. This shows how easily the same pipeline can be implemented using LlamaIndex.


In [ ]:
import chromadb
from chromadb import Settings
from llama_index.llms.openai import OpenAI
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore

# ChromaDB Vector Store
chroma_client = chromadb.PersistentClient(
    path="./data/baseline-rag/chromadb", settings=Settings(allow_reset=True))
chroma_client.reset()
collection = chroma_client.get_or_create_collection(
    name="malazan", metadata={"hnsw:space": "cosine"})
vector_store = ChromaVectorStore(chroma_collection=collection)

# OpenAI Embedding and LLM
embedding = OpenAIEmbedding(api_key=OPENAI_API_KEY,
                            model="text-embedding-3-small")
llm = OpenAI(api_key=OPENAI_API_KEY, model="gpt-4-turbo")

# Define the ingestion pipeline to add documents to vector store
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=512, chunk_overlap=20),
        embedding,
    ],
    vector_store=vector_store,
)

# Create index with the vector store and using the embedding model
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, embed_model=embedding)

In [ ]:
# Fetch documents
documents = SimpleDirectoryReader('./data/docs').load_data()

# Run pipeline
pipeline.run(documents=documents)

#### Create base QueryEngine from LlamaIndex


In [ ]:
query_engine = index.as_query_engine(llm=llm, verbose=True)

#### Or alternatively, create a CustomQueryEngine


In [ ]:
from llama_index.core import PromptTemplate
from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.retrievers import BaseRetriever
from llama_index.core import get_response_synthesizer
from llama_index.core.response_synthesizers import BaseSynthesizer

qa_prompt = PromptTemplate(
    """You are a helpful assistant that answers questions about the Malazan Fantasy Universe using provided context.
    Context information is below.
    ---------------------
    {context_str}
    ---------------------
    Given the context information and not prior knowledge, answer the query.
    Query: {query_str}
    Answer: 
    """,
)


class RAGQueryEngine(CustomQueryEngine):
    """RAG String Query Engine."""

    retriever: BaseRetriever
    response_synthesizer: BaseSynthesizer
    llm: OpenAI
    qa_prompt: PromptTemplate

    def custom_query(self, query_str: str):
        nodes = self.retriever.retrieve(query_str)
        context_str = "\n\n".join([n.node.get_content() for n in nodes])
        print("Prompt:\n\n", qa_prompt.format(
            context_str=context_str, query_str=query_str))
        response = self.llm.complete(
            qa_prompt.format(context_str=context_str, query_str=query_str)
        )

        return str(response)


synthesizer = get_response_synthesizer(response_mode="compact")
query_engine = RAGQueryEngine(
    retriever=index.as_retriever(),
    response_synthesizer=synthesizer,
    llm=llm,
    qa_prompt=qa_prompt,
)

In [ ]:
response = query_engine.query(query)
display(Markdown(f"{response}"))

## Simplest RAG implementation using LlamaIndex


In [15]:
# Fetch documents
documents = SimpleDirectoryReader('./data/docs').load_data()

# build VectorStoreIndex that takes care of chunking documents
# and encoding chunks to embeddings for future retrieval
index = VectorStoreIndex.from_documents(documents=documents)

# The QueryEngine class is equipped with the generator
# and facilitates the retrieval and generation steps
query_engine = index.as_query_engine()

# Use your Default RAG
response = query_engine.query(query)
display(Markdown(f"{response}"))

Anomander Rake's titles included Lord of Moon's Spawn, Son of Darkness, Knight of Darkness, Black-Winged Lord, Blacksword, First Son of Darkness, Knight of High House Dark, The Mane of Chaos, The Rake, and Anomandaris Irake, Anomandaris Purake, Anomander Dragnipurake.